In [13]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import requests
import time
import random

In [63]:
df = pd.read_csv('data_sample/news_prices_full.csv')
df.head()

,date_publish,date_download,source_domain,news_outlet,title,description,maintext,url,language,sentiment_negative,...,emotion_surprise,emotion_neutral,named_entities,mentioned_companies,related_companies,industries,ticker,prev_day_price,curr_day_price,next_day_price
0,2019-08-03 00:00:00,2019-09-03 17:40:27+00:00,finance.yahoo.com,finance.yahoo.com,Google’s new Android Focus Mode could break yo...,Google's new Focus Mode feature for Android is...,I have an unhealthy addiction to a number of a...,https://finance.yahoo.com/news/android-focus-m...,en,0.000325,...,0.019017,0.805375,"[""Google"", ""GOOG"", ""GOOGL"", ""Focus Mode"", ""Goo...","[""GOOGL"", ""META""]","[""IGLD"", ""RAMP"", ""NSR"", ""TWTR"", ""ACXM"", ""COR"",...",[7375],GOOGL,NaN,NaN,NaN
1,2019-08-03 00:00:00,2019-09-03 17:40:27+00:00,finance.yahoo.com,finance.yahoo.com,Google’s new Android Focus Mode could break yo...,Google's new Focus Mode feature for Android is...,I have an unhealthy addiction to a number of a...,https://finance.yahoo.com/news/android-focus-m...,en,0.000325,...,0.019017,0.805375,"[""Google"", ""GOOG"", ""GOOGL"", ""Focus Mode"", ""Goo...","[""GOOGL"", ""META""]","[""IGLD"", ""RAMP"", ""NSR"", ""TWTR"", ""ACXM"", ""COR"",...",[7375],META,NaN,NaN,NaN
2,2019-01-22 07:29:49,2019-01-22 08:20:01+00:00,finance.yahoo.com,finance.yahoo.com,UBS Warns on Client Activity After $13 Billion...,Withdrawals at the Zurich-based bank’s key glo...,(Bloomberg) -- UBS Group AG warned client acti...,https://finance.yahoo.com/news/ubs-sees-outflo...,en,0.997143,...,0.022429,0.071112,"[""Zurich"", ""Andrew Coombs"", ""Citigroup"", ""Serg...","[""C""]","[""BSAC"", ""FHN"", ""PACW"", ""BSMX"", ""VLY"", ""MBRG"",...",[6021],C,63.12,61.85000,62.13
3,2019-08-01 13:03:23,2019-08-01 13:23:12+00:00,finance.yahoo.com,finance.yahoo.com,Shopify Boosts Outlook on New Online Offerings,(Bloomberg) -- Shopify Inc. shares continued t...,(Bloomberg) -- Shopify Inc. shares continued t...,https://finance.yahoo.com/news/shopify-boosts-...,en,0.000223,...,0.107670,0.752265,"[""Canadian"", ""Shopify"", ""New York"", ""Amy Shape...","[""SHOP""]","[""WEB"", ""PTE"", ""AMWL"", ""NLOK"", ""GSX"", ""YOU"", ""...",[7372],SHOP,317.88,341.39001,332.19
4,2019-10-31 13:48:15,2019-10-31 14:39:52+00:00,finance.yahoo.com,finance.yahoo.com,Wayfair Plunges as Forecast Miss Heightens Gro...,(Bloomberg) -- Wayfair Inc. plunged near its l...,(Bloomberg) -- Wayfair Inc. plunged near its l...,https://finance.yahoo.com/news/wayfair-plunges...,en,0.998786,...,0.080362,0.254296,"[""New York"", ""Boston"", ""Citi"", ""Nicholas Jones...","[""C""]","[""BSAC"", ""FHN"", ""PACW"", ""BSMX"", ""VLY"", ""MBRG"",...",[6021],C,72.97,71.86000,73.84


In [64]:
df.shape

(100428, 27)

In [65]:
df.isnull().sum()

date_publish               0
date_download              0
source_domain              0
news_outlet                0
title                      0
description               35
maintext                   0
url                        0
language                   0
sentiment_negative         0
sentiment_neutral          0
sentiment_positive         0
emotion_anger              0
emotion_fear               0
emotion_joy                0
emotion_sadness            0
emotion_disgust            0
emotion_surprise           0
emotion_neutral            0
named_entities             0
mentioned_companies        0
related_companies          0
industries                 0
ticker                     0
prev_day_price         14461
curr_day_price         14451
next_day_price         14461
dtype: int64

### Get all tickers where prices can be null

In [66]:
null_prices = df.loc[(df['prev_day_price'].isnull()) |  (df['curr_day_price'].isnull()) | (df['next_day_price'].isnull())]
null_prices.head()

,date_publish,date_download,source_domain,news_outlet,title,description,maintext,url,language,sentiment_negative,...,emotion_surprise,emotion_neutral,named_entities,mentioned_companies,related_companies,industries,ticker,prev_day_price,curr_day_price,next_day_price
0,2019-08-03 00:00:00,2019-09-03 17:40:27+00:00,finance.yahoo.com,finance.yahoo.com,Google’s new Android Focus Mode could break yo...,Google's new Focus Mode feature for Android is...,I have an unhealthy addiction to a number of a...,https://finance.yahoo.com/news/android-focus-m...,en,0.000325,...,0.019017,0.805375,"[""Google"", ""GOOG"", ""GOOGL"", ""Focus Mode"", ""Goo...","[""GOOGL"", ""META""]","[""IGLD"", ""RAMP"", ""NSR"", ""TWTR"", ""ACXM"", ""COR"",...",[7375],GOOGL,NaN,NaN,NaN
1,2019-08-03 00:00:00,2019-09-03 17:40:27+00:00,finance.yahoo.com,finance.yahoo.com,Google’s new Android Focus Mode could break yo...,Google's new Focus Mode feature for Android is...,I have an unhealthy addiction to a number of a...,https://finance.yahoo.com/news/android-focus-m...,en,0.000325,...,0.019017,0.805375,"[""Google"", ""GOOG"", ""GOOGL"", ""Focus Mode"", ""Goo...","[""GOOGL"", ""META""]","[""IGLD"", ""RAMP"", ""NSR"", ""TWTR"", ""ACXM"", ""COR"",...",[7375],META,NaN,NaN,NaN
28,2019-08-10 03:03:44,2019-08-10 04:15:31+00:00,finance.yahoo.com,finance.yahoo.com,Apple offers $1 million reward to anyone who c...,Apple is expanding its bug bounty program — an...,"This article, Apple offers $1 million reward t...",https://finance.yahoo.com/news/apple-offers-1-...,en,0.000503,...,0.041337,0.897683,"[""Apple"", ""iPhone"", ""Apple"", ""iOS"", ""Apple"", ""...","[""AAPL"", ""GOOGL""]","[""IGLD"", ""TDC"", ""CRAY"", ""TCX"", ""SMCI"", ""YHOO"",...","[3571, 7375]",AAPL,NaN,NaN,NaN
29,2019-08-10 03:03:44,2019-08-10 04:15:31+00:00,finance.yahoo.com,finance.yahoo.com,Apple offers $1 million reward to anyone who c...,Apple is expanding its bug bounty program — an...,"This article, Apple offers $1 million reward t...",https://finance.yahoo.com/news/apple-offers-1-...,en,0.000503,...,0.041337,0.897683,"[""Apple"", ""iPhone"", ""Apple"", ""iOS"", ""Apple"", ""...","[""AAPL"", ""GOOGL""]","[""IGLD"", ""TDC"", ""CRAY"", ""TCX"", ""SMCI"", ""YHOO"",...","[3571, 7375]",GOOGL,NaN,NaN,NaN
32,2019-01-20 02:40:01,2019-01-20 04:02:21+00:00,finance.yahoo.com,finance.yahoo.com,Italy Is Said to Reject Telecom Italia's Grid ...,"The plan, proposed last year by former Chief E...",(Bloomberg) -- Italy’s communications regulato...,https://finance.yahoo.com/news/italy-said-reje...,en,0.788156,...,0.011818,0.805146,"[""Italy"", ""Amos Genish"", ""Milan"", ""Luigi Gubit...","[""C"", ""V""]","[""FIS"", ""BSAC"", ""FRXB"", ""MA"", ""FHN"", ""AAQC"", ""...","[7389, 6021]",C,NaN,NaN,NaN


In [67]:
null_prices.shape

(14471, 27)

In [68]:
null_tickers = list(null_prices['ticker'].unique())
len(null_tickers)

49

In [69]:
min_date, max_date = null_prices['date_publish'].min(), null_prices['date_publish'].max()
print(f'Min date: {min_date}, Max date: {max_date}')

Min date: 2017-01-03 00:00:00, Max date: 2023-12-30 18:15:00


In [70]:
start = datetime(2017, 1, 1)
end = datetime.today()

In [71]:
tickers = [ticker.replace('.', '-') + '.US' for ticker in null_tickers]
tickers[:5]

['GOOGL.US', 'META.US', 'AAPL.US', 'C.US', 'V.US']

### Get stock prices for the tickers having null prices

In [23]:
tickers_errors = []

In [24]:
def fetch_ticker_data(ticker):
    """Downloads data with a polite delay and jitter."""
    # Adding jitter: sleep between 3 and 7 seconds
    # This prevents 'bursty' traffic where all threads wake up at once
    delay = 5 + random.uniform(-2, 2)
    print(f"Waiting {delay:.2f}s before fetching {ticker}...")
    time.sleep(delay)
    url = f"https://stooq.com/q/d/l/?s={ticker}&d1={start:%Y%m%d}&d2={end:%Y%m%d}&i=d"
    try:
        df = pd.read_csv(url, parse_dates=["Date"])
        if df.empty:
            print(f"Warning: No data for {ticker}")
            return None
        df["Ticker"] = ticker.split('.')[0].upper() 
        return df
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        tickers_errors.append(ticker)
        return None

In [25]:
with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(fetch_ticker_data, tickers))

dfs = [df for df in results if df is not None]

Waiting 6.98s before fetching GOOGL.US...
Waiting 3.36s before fetching META.US...
Waiting 6.70s before fetching AAPL.US...
Waiting 4.03s before fetching C.US...
Waiting 3.70s before fetching V.US...
Waiting 6.95s before fetching T.US...
Waiting 4.50s before fetching BA.US...
Waiting 6.68s before fetching NFLX.US...
Waiting 3.03s before fetching MSFT.US...
Waiting 6.28s before fetching JPM.US...
Waiting 3.83s before fetching BABA.US...
Waiting 6.26s before fetching TSLA.US...
Waiting 6.30s before fetching GS.US...
Waiting 5.19s before fetching INTC.US...
Waiting 3.31s before fetching BRK.US...
Waiting 4.95s before fetching AMZN.US...
Error fetching BRK.US: Missing column provided to 'parse_dates': 'Date'
Waiting 6.02s before fetching CVX.US...
Waiting 4.66s before fetching XOM.US...
Waiting 4.19s before fetching PFE.US...
Waiting 3.44s before fetching MA.US...
Waiting 6.03s before fetching WMT.US...
Waiting 4.54s before fetching GE.US...
Waiting 6.88s before fetching QCOM.US...
Waiting

In [27]:
tickers_errors

['BRK.US', 'SQ.US']

In [28]:
# видно не судьба, видно не судьба ...
with ThreadPoolExecutor(max_workers=3) as executor:
    results2 = list(executor.map(fetch_ticker_data, tickers_errors))

Waiting 4.47s before fetching BRK.US...
Waiting 6.85s before fetching SQ.US...
Error fetching BRK.US: Missing column provided to 'parse_dates': 'Date'
Error fetching SQ.US: Missing column provided to 'parse_dates': 'Date'


In [26]:
final_df = (pd.concat(dfs, ignore_index=True)
                .set_index(["Ticker", "Date"])
                .sort_index())
final_df = final_df.reset_index()
final_df

,Ticker,Date,Open,High,Low,Close,Volume
0,AAPL,2017-01-03,26.6898,26.8154,26.4434,26.7722,120355375.0
1,AAPL,2017-01-04,26.7024,26.8576,26.6798,26.7428,89031733.0
2,AAPL,2017-01-05,26.7192,26.9361,26.6927,26.8812,94131019.0
3,AAPL,2017-01-06,26.9186,27.2369,26.8479,27.1759,134161434.0
4,AAPL,2017-01-09,27.1846,27.5304,27.1818,27.4244,137261843.0
...,...,...,...,...,...,...,...
106433,XOM,2026-02-03,138.6400,145.0100,138.3100,143.7300,37577844.0
106434,XOM,2026-02-04,144.1150,147.8400,144.1150,147.5900,29426241.0
106435,XOM,2026-02-05,146.5800,146.7300,143.7950,146.0800,18431739.0
106436,XOM,2026-02-06,146.6200,149.5700,146.0900,149.0500,17216025.0


In [29]:
final_df.to_csv('data_sample/prices_for_merging.csv', index=False)

In [72]:
final_df = pd.read_csv('data_sample/prices_for_merging.csv')

In [73]:
final_df['Date'] = pd.to_datetime(final_df['Date'], errors='coerce', infer_datetime_format=True)
final_df['date'] = final_df['Date'].dt.normalize()
final_df['day_of_week'] = final_df['date'].dt.day_name()
final_df

,Ticker,Date,Open,High,Low,Close,Volume,date,day_of_week
0,AAPL,2017-01-03,26.6898,26.8154,26.4434,26.7722,120355375.0,2017-01-03,Tuesday
1,AAPL,2017-01-04,26.7024,26.8576,26.6798,26.7428,89031733.0,2017-01-04,Wednesday
2,AAPL,2017-01-05,26.7192,26.9361,26.6927,26.8812,94131019.0,2017-01-05,Thursday
3,AAPL,2017-01-06,26.9186,27.2369,26.8479,27.1759,134161434.0,2017-01-06,Friday
4,AAPL,2017-01-09,27.1846,27.5304,27.1818,27.4244,137261843.0,2017-01-09,Monday
...,...,...,...,...,...,...,...,...,...
106433,XOM,2026-02-03,138.6400,145.0100,138.3100,143.7300,37577844.0,2026-02-03,Tuesday
106434,XOM,2026-02-04,144.1150,147.8400,144.1150,147.5900,29426241.0,2026-02-04,Wednesday
106435,XOM,2026-02-05,146.5800,146.7300,143.7950,146.0800,18431739.0,2026-02-05,Thursday
106436,XOM,2026-02-06,146.6200,149.5700,146.0900,149.0500,17216025.0,2026-02-06,Friday


In [74]:
final_df['day_of_week'].value_counts()

Tuesday      21911
Wednesday    21771
Friday       21446
Thursday     21396
Monday       19914
Name: day_of_week, dtype: int64

In [75]:
def convert_publish_dates(df, col='date_publish'):
    dt = pd.to_datetime(df[col])
    date_dt = dt.dt.normalize()
    df['date'] = date_dt
    df['prev_date'] = (date_dt - pd.Timedelta(days=1))
    df['future_date'] = (date_dt + pd.Timedelta(days=1))
    df['date_day_of_week'] = date_dt.dt.day_name()
    df['prev_day_of_week'] = (date_dt - pd.Timedelta(days=1)).dt.day_name()
    df['future_day_of_week'] = (date_dt + pd.Timedelta(days=1)).dt.day_name()
    return df

In [76]:
df = convert_publish_dates(df)
df

,date_publish,date_download,source_domain,news_outlet,title,description,maintext,url,language,sentiment_negative,...,ticker,prev_day_price,curr_day_price,next_day_price,date,prev_date,future_date,date_day_of_week,prev_day_of_week,future_day_of_week
0,2019-08-03 00:00:00,2019-09-03 17:40:27+00:00,finance.yahoo.com,finance.yahoo.com,Google’s new Android Focus Mode could break yo...,Google's new Focus Mode feature for Android is...,I have an unhealthy addiction to a number of a...,https://finance.yahoo.com/news/android-focus-m...,en,0.000325,...,GOOGL,NaN,NaN,NaN,2019-08-03,2019-08-02,2019-08-04,Saturday,Friday,Sunday
1,2019-08-03 00:00:00,2019-09-03 17:40:27+00:00,finance.yahoo.com,finance.yahoo.com,Google’s new Android Focus Mode could break yo...,Google's new Focus Mode feature for Android is...,I have an unhealthy addiction to a number of a...,https://finance.yahoo.com/news/android-focus-m...,en,0.000325,...,META,NaN,NaN,NaN,2019-08-03,2019-08-02,2019-08-04,Saturday,Friday,Sunday
2,2019-01-22 07:29:49,2019-01-22 08:20:01+00:00,finance.yahoo.com,finance.yahoo.com,UBS Warns on Client Activity After $13 Billion...,Withdrawals at the Zurich-based bank’s key glo...,(Bloomberg) -- UBS Group AG warned client acti...,https://finance.yahoo.com/news/ubs-sees-outflo...,en,0.997143,...,C,63.12,61.85000,62.13000,2019-01-22,2019-01-21,2019-01-23,Tuesday,Monday,Wednesday
3,2019-08-01 13:03:23,2019-08-01 13:23:12+00:00,finance.yahoo.com,finance.yahoo.com,Shopify Boosts Outlook on New Online Offerings,(Bloomberg) -- Shopify Inc. shares continued t...,(Bloomberg) -- Shopify Inc. shares continued t...,https://finance.yahoo.com/news/shopify-boosts-...,en,0.000223,...,SHOP,317.88,341.39001,332.19000,2019-08-01,2019-07-31,2019-08-02,Thursday,Wednesday,Friday
4,2019-10-31 13:48:15,2019-10-31 14:39:52+00:00,finance.yahoo.com,finance.yahoo.com,Wayfair Plunges as Forecast Miss Heightens Gro...,(Bloomberg) -- Wayfair Inc. plunged near its l...,(Bloomberg) -- Wayfair Inc. plunged near its l...,https://finance.yahoo.com/news/wayfair-plunges...,en,0.998786,...,C,72.97,71.86000,73.84000,2019-10-31,2019-10-30,2019-11-01,Thursday,Wednesday,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100423,2021-02-17 15:25:53,2021-02-17 15:41:21+00:00,finance.yahoo.com,finance.yahoo.com,'No Man's Sky' update lets players adopt alien...,You can ride giant creatures and companions ca...,Benzinga\nWarren Buffett's Berkshire Cuts Appl...,https://finance.yahoo.com/news/no-mans-sky-com...,en,0.004761,...,AAPL,133.19,130.84000,129.71001,2021-02-17,2021-02-16,2021-02-18,Wednesday,Tuesday,Thursday
100424,2021-02-17 15:25:53,2021-02-17 15:41:21+00:00,finance.yahoo.com,finance.yahoo.com,'No Man's Sky' update lets players adopt alien...,You can ride giant creatures and companions ca...,Benzinga\nWarren Buffett's Berkshire Cuts Appl...,https://finance.yahoo.com/news/no-mans-sky-com...,en,0.004761,...,CVX,93.13,95.92000,95.00000,2021-02-17,2021-02-16,2021-02-18,Wednesday,Tuesday,Thursday
100425,2021-02-17 15:25:53,2021-02-17 15:41:21+00:00,finance.yahoo.com,finance.yahoo.com,'No Man's Sky' update lets players adopt alien...,You can ride giant creatures and companions ca...,Benzinga\nWarren Buffett's Berkshire Cuts Appl...,https://finance.yahoo.com/news/no-mans-sky-com...,en,0.004761,...,MRK,74.25,75.54000,75.41000,2021-02-17,2021-02-16,2021-02-18,Wednesday,Tuesday,Thursday
100426,2021-02-17 15:25:53,2021-02-17 15:41:21+00:00,finance.yahoo.com,finance.yahoo.com,'No Man's Sky' update lets players adopt alien...,You can ride giant creatures and companions ca...,Benzinga\nWarren Buffett's Berkshire Cuts Appl...,https://finance.yahoo.com/news/no-mans-sky-com...,en,0.004761,...,BRK,245.28,370500.00000,245.25000,2021-02-17,2021-02-16,2021-02-18,Wednesday,Tuesday,Thursday


In [77]:
def fill_price(main_df, target_col, date_key, price_type):
    """
    main_df: Your primary dataframe
    target_col: The column to fill (e.g., 'prev_day_price')
    date_key: The date column in main_df to match on (e.g., 'prev_date')
    price_type: 'close' or 'open' from prices_df
    """
    # Create a lookup mapping from the prices dataframe
    # We use a multi-index (Ticker, date) for a precise match
    lookup = final_df.set_index(['Ticker', 'date'])[price_type]
    # Identify where target_col is NaN
    mask = main_df[target_col].isna()
    # Map the values: create a temporary series of keys from the main_df rows that need filling
    keys = pd.MultiIndex.from_frame(main_df.loc[mask, ['ticker', date_key]])
    # Fill only the missing spots using the mapping
    main_df.loc[mask, target_col] = keys.map(lookup)

In [78]:
# Fill prev_day_price (using Close)
fill_price(df, 'prev_day_price', 'prev_date', 'Close')
# Fill curr_day_price (using Close)
fill_price(df, 'curr_day_price', 'date', 'Close')
# Fill next_day_price (using Open)
fill_price(df, 'next_day_price', 'future_date', 'Open')
print("Missing values filled where price data was available.")

Missing values filled where price data was available.


In [79]:
df.isnull().sum()

date_publish               0
date_download              0
source_domain              0
news_outlet                0
title                      0
description               35
maintext                   0
url                        0
language                   0
sentiment_negative         0
sentiment_neutral          0
sentiment_positive         0
emotion_anger              0
emotion_fear               0
emotion_joy                0
emotion_sadness            0
emotion_disgust            0
emotion_surprise           0
emotion_neutral            0
named_entities             0
mentioned_companies        0
related_companies          0
industries                 0
ticker                     0
prev_day_price          6315
curr_day_price         11033
next_day_price          6509
date                       0
prev_date                  0
future_date                0
date_day_of_week           0
prev_day_of_week           0
future_day_of_week         0
dtype: int64

In [81]:
pd.set_option('display.max_columns', None)
df.head(3)

,date_publish,date_download,source_domain,news_outlet,title,description,maintext,url,language,sentiment_negative,sentiment_neutral,sentiment_positive,emotion_anger,emotion_fear,emotion_joy,emotion_sadness,emotion_disgust,emotion_surprise,emotion_neutral,named_entities,mentioned_companies,related_companies,industries,ticker,prev_day_price,curr_day_price,next_day_price,date,prev_date,future_date,date_day_of_week,prev_day_of_week,future_day_of_week
0,2019-08-03 00:00:00,2019-09-03 17:40:27+00:00,finance.yahoo.com,finance.yahoo.com,Google’s new Android Focus Mode could break yo...,Google's new Focus Mode feature for Android is...,I have an unhealthy addiction to a number of a...,https://finance.yahoo.com/news/android-focus-m...,en,0.000325,0.999305,0.000370,0.056825,0.006890,0.002060,0.007182,0.10265,0.019017,0.805375,"[""Google"", ""GOOG"", ""GOOGL"", ""Focus Mode"", ""Goo...","[""GOOGL"", ""META""]","[""IGLD"", ""RAMP"", ""NSR"", ""TWTR"", ""ACXM"", ""COR"",...",[7375],GOOGL,59.5999,NaN,NaN,2019-08-03,2019-08-02,2019-08-04,Saturday,Friday,Sunday
1,2019-08-03 00:00:00,2019-09-03 17:40:27+00:00,finance.yahoo.com,finance.yahoo.com,Google’s new Android Focus Mode could break yo...,Google's new Focus Mode feature for Android is...,I have an unhealthy addiction to a number of a...,https://finance.yahoo.com/news/android-focus-m...,en,0.000325,0.999305,0.000370,0.056825,0.006890,0.002060,0.007182,0.10265,0.019017,0.805375,"[""Google"", ""GOOG"", ""GOOGL"", ""Focus Mode"", ""Goo...","[""GOOGL"", ""META""]","[""IGLD"", ""RAMP"", ""NSR"", ""TWTR"", ""ACXM"", ""COR"",...",[7375],META,189.0200,NaN,NaN,2019-08-03,2019-08-02,2019-08-04,Saturday,Friday,Sunday
2,2019-01-22 07:29:49,2019-01-22 08:20:01+00:00,finance.yahoo.com,finance.yahoo.com,UBS Warns on Client Activity After $13 Billion...,Withdrawals at the Zurich-based bank’s key glo...,(Bloomberg) -- UBS Group AG warned client acti...,https://finance.yahoo.com/news/ubs-sees-outflo...,en,0.997143,0.000320,0.002537,0.023659,0.037095,0.002821,0.799185,0.04370,0.022429,0.071112,"[""Zurich"", ""Andrew Coombs"", ""Citigroup"", ""Serg...","[""C""]","[""BSAC"", ""FHN"", ""PACW"", ""BSMX"", ""VLY"", ""MBRG"",...",[6021],C,63.1200,61.85,62.13,2019-01-22,2019-01-21,2019-01-23,Tuesday,Monday,Wednesday


In [82]:
df.columns

Index(['date_publish', 'date_download', 'source_domain', 'news_outlet',
       'title', 'description', 'maintext', 'url', 'language',
       'sentiment_negative', 'sentiment_neutral', 'sentiment_positive',
       'emotion_anger', 'emotion_fear', 'emotion_joy', 'emotion_sadness',
       'emotion_disgust', 'emotion_surprise', 'emotion_neutral',
       'named_entities', 'mentioned_companies', 'related_companies',
       'industries', 'ticker', 'prev_day_price', 'curr_day_price',
       'next_day_price', 'date', 'prev_date', 'future_date',
       'date_day_of_week', 'prev_day_of_week', 'future_day_of_week'],
      dtype='object')

In [84]:
df = df[['date', 'prev_date', 'future_date', 'title', 'description', 'maintext',
         'ticker', 'prev_day_price', 'curr_day_price', 'next_day_price',
         'named_entities', 'mentioned_companies', 'related_companies',
         'sentiment_negative', 'sentiment_neutral', 'sentiment_positive',
       'emotion_anger', 'emotion_fear', 'emotion_joy', 'emotion_sadness',
       'emotion_disgust', 'emotion_surprise', 'emotion_neutral',
       'date_day_of_week', 'prev_day_of_week', 'future_day_of_week',
         'language']]

In [86]:
df.to_parquet('data_sample/news_prices_full_processed.parquet', index=False)